In [ ]:
### Import required libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

import os

In [ ]:
### Make sure that 'ggplot' style is used for all plots
plt.style.use('ggplot')
# plt.style.available ### To view all other available styles

In [ ]:
### Set Working Directory (WD)
os.chdir('/Volumes/GoogleDrive/My Drive/CEMEX/Data Translators/GitHub/rgamerosl/capstone-project')

In [ ]:
### Read the data
df = pd.read_excel("dataset/data_v0.xlsx")

In [ ]:
df.info()

In [ ]:
df.iloc[-1,]

In [ ]:
### For some strange reason when reading the Excel file it detects 1'021,336 entries. However I know there are only 466,786 differente entries, everything else are just empty rows
df = df.iloc[0:466786,0:18]
df.info()

In [ ]:
df_backup = df.copy(deep=True)

In [ ]:
display(df.head(7))

### Considering that "liters_per_hour" is simple the division between "Fuel_used" and "Engine_hrs" then I should exclude these two columns from my model
### Also as I see it the "Drive_time" is only transform the "Engine_hrs" into hh:mm:ss format, therefore it should also be ignored.

In [ ]:
df['liters_per_hour'].describe()

In [ ]:
### First replace empty strings with nan
df['km_per_liter'] = df['km_per_liter'].replace(r'^\s*$', np.nan, regex=True)
### Transform strings into float numbers
df['km_per_liter'] = df['km_per_liter'].astype(float)

In [ ]:
### First replace empty strings with nan
df['Hrs_eff'] = df['Hrs_eff'].replace(r'^\s*$', np.nan, regex=True)
### Transform strings into float numbers
df['Hrs_eff'] = df['Hrs_eff'].astype(float)

In [ ]:
### First replace empty strings with nan
df['liters_per_hour'] = df['liters_per_hour'].replace(r'^\s*$', np.nan, regex=True)
### Transform strings into float numbers
df['liters_per_hour'] = df['liters_per_hour'].astype(float)

In [ ]:
df['liters_per_hour'].describe()

In [ ]:
plt.hist(df['liters_per_hour'])
plt.show()

In [ ]:
### Save dataset with outliers for liters_per_hour
# outliers1 = df.loc[abs(df['liters_per_hour']) > 20,]
# outliers2 = df.loc[df['liters_per_hour'] < 0,]
# outliers = pd.concat([outliers1, outliers2])
# # outliers.to_excel("dataset/outliers_lts.xlsx")

In [ ]:
### Save dataset with outliers for Engine_hrs and Idle_time
# outliers_1 = df.loc[abs(df['Engine_hrs']) > 24,]
# outliers_2 = df.loc[abs(df["Idle_time"]) > 1,]
# outliers_ = pd.concat([outliers_1, outliers_2])
# # outliers_.to_excel("dataset/outliers_hrs.xlsx")

In [ ]:
### Everything bigger than 20 should be nan
df.loc[abs(df['liters_per_hour']) > 20,'liters_per_hour'] = np.nan
### Also every negative value should be nan
df.loc[df['liters_per_hour'] < 0,'liters_per_hour'] = np.nan

In [ ]:
plt.hist(df['liters_per_hour'], density=True, bins=20)
# plt.savefig('figures/histogram-liters_per_hour.png')
plt.show()

In [ ]:
df['liters_per_hour'].describe()

In [ ]:
### Amount of nan in the liters_per_hour (target value)
df['liters_per_hour'].isna().sum()/len(df['liters_per_hour'])

In [ ]:
df['Manufacturer'].unique()

In [ ]:
### Find out how many observations we have from each manufacturer
df.Manufacturer.value_counts()

In [ ]:
### Group different spellings for the same manufacturer

df.loc[df['Manufacturer']=="INTERNATIONAL","Manufacturer"] = "International"
df.loc[df['Manufacturer']=="FREIGHTLINER","Manufacturer"] = "Freightliner"
df.loc[df['Manufacturer']=="freightliner","Manufacturer"] = "Freightliner"
df.loc[df['Manufacturer']=="MAN","Manufacturer"] = "Man"

In [ ]:
### Count the amount of differents trucks analysed by Manufacturers
print(len(df_backup['Plate'].unique()))
df.groupby('Manufacturer').Plate.nunique() 

### Something could be wrong as there are only 2330 different Plates numbers, however when doing the counts by manufacturer I get 2592 Plates.

In [ ]:
Plates = df.groupby('Plate').Manufacturer.nunique()
display(Plates.head(10))

In [ ]:
len(Plates[Plates > 1])
display(Plates[Plates > 1])

# Plates[Plates > 1].to_excel("dataset/CRs_with_2_Manufacturers.xlsx")

### There are 262 Plates numbers that have listed 2 different manufacturers, what should we do with those?
### Maybe look for an extra dataset only with the Plate number and Manufacturer to validate the real value for the Manufacturer of these 262 Trucks

In [ ]:
CRs = df_backup[df_backup["Plate"].isin(Plates[Plates > 1].index.values)]
len(CRs['Plate'].unique())

# CRs.to_excel("dataset/CRs_multiple_manufacturers.xlsx")

In [ ]:
df.loc[df["Plate"]=="CR2856","Manufacturer"].unique()

In [ ]:
### Considering that there are only 1 truck from the following Manufactrers: Astra, Scania and Volvo, I think we could get rid of these 3 trucks for the analysis
df2 = df[~df.Manufacturer.isin(["Astra", "Scania", "Volvo"])]
print(df2.Manufacturer.value_counts())
print(df2.groupby('Manufacturer').Plate.nunique())

In [ ]:
df2.info()

In [ ]:
df2.loc[0,'Date']

In [ ]:
### Add column for Weekdays (0: Monday to 6: Sunday)
df2['Weekday'] = pd.to_datetime(df2['Date']).dt.dayofweek

In [ ]:
df2.loc[1,]

In [ ]:
df2['Weekday'].unique()

In [ ]:
df2.Weekday.value_counts().sort_index()

In [ ]:
df2['Idle_time'].describe()
### It looks to me like Idle_time should always be a value between 0 and 1, so maybe everything bigger than 1 could be a typo (should it be divided by 24 maybe?)

In [ ]:
df2['TurnOn'].describe()
### Higher values seems very unlikely, what to do with them?

In [ ]:
df2['Engine_hrs'].describe()
### How could you have Engine_hrs bigger than 24 in just one day?

In [ ]:
df2.loc[abs(df2['Engine_hrs']) > 24,].shape


In [ ]:
df2.loc[abs(df2['Idle_time']) > 1,].shape

In [ ]:
### Delete entries with Engine_hrs bigger than 24
indexEngine = df2.loc[abs(df2['Engine_hrs'] > 24),].index
df2.drop(indexEngine,inplace=True)
df2.info()

In [ ]:
plt.hist(df2['Engine_hrs'], density=True, bins=24)
# plt.savefig('figures/histogram-Engine_hrs.png')
plt.show()

In [ ]:
### Delete entries with Idle_time bigger than 1
indexIdle = df2.loc[abs(df2['Idle_time'] > 1),].index
df2.drop(indexIdle,inplace=True)
df2.info()

### Asks if delition is the way to go... maybe just leave those entries with pd.nan values could work
### The Engine_hrs I do believe need to be deleted as they are very few and they impact directly on the calculation of the target variable (liters_per_hour)
### But inn the Idle_time case, since is only a feature, maybe put them with pd.nan values could work

In [ ]:
df2['Idle_time'].describe()

In [ ]:
### Transform Idle_time to hours (multiply by 24)
df2['Idle_time'] = df2['Idle_time']*24
df2['Idle_time'].describe()

In [ ]:
plt.hist(df2['Idle_time'], density=True, bins=24)
# plt.savefig('figures/histogram-Idle_time.png')
plt.show()

In [ ]:
df2.info()

In [ ]:
plt.hist(df2['Fuel_used'], density=True)
plt.show()

In [ ]:
df2['Fuel_used'].describe()

In [ ]:
indexFuelNeg = df2.loc[df2['Fuel_used'] < 0,].index
df2.drop(indexFuelNeg,inplace=True)
indexFuelHigh = df2.loc[df2['Fuel_used'] > 200,].index
df2.drop(indexFuelHigh,inplace=True)

df2.info()

In [ ]:
plt.hist(df2['Fuel_used'], density=True, bins=20)
# plt.savefig('figures/histogram-Fuel_used.png')
plt.show()

In [ ]:
manufacturer_catalog = pd.read_excel("dataset/Catalog.xlsx")
manufacturer_catalog.info()

In [ ]:
manufacturer_catalog.columns=["Plate", "Year_Model2", "Manufacturer2"]

In [ ]:
display(manufacturer_catalog.head(10))

In [ ]:
df3 = df2.merge(manufacturer_catalog,on='Plate',how="left")
display(df3.head(10))

In [ ]:
df3.info()

In [ ]:
Plates2 = df3.groupby('Plate').Manufacturer.nunique()
Plates2[Plates2 > 1]

In [ ]:
Plates3 = df3.groupby('Plate').Manufacturer2.nunique()
Plates3[Plates3 > 1]

In [ ]:
df3.loc[df3["Plate"]=="CR5395","Manufacturer"].unique()

In [ ]:
df3.loc[df3["Plate"]=="CR5395","Manufacturer2"].unique()

In [ ]:
df3['Manufacturer'] = df3['Manufacturer2']
df3['Year_Model'] = df3['Year_Model2']

df4 = df3.iloc[0:450224,0:19]

In [ ]:
df4.info()

In [ ]:
df4['TurnOn'].describe()

In [ ]:
plt.hist(df4['TurnOn'], density=True, bins=20)
plt.show()

In [ ]:
### Put a threshold for the amount of times the motor of each truck is TurnOn along the day
df4[df4['TurnOn']>20]['TurnOn'].count()

In [ ]:
df4[df4['TurnOn']<=20]['TurnOn'].describe()

In [ ]:
plt.hist(df4[df4['TurnOn']<=20]['TurnOn'], density=True, bins=20)
# plt.savefig('figures/histogram-TurnOn.png')
plt.show()

In [ ]:
### Delete entries with TurnOn bigger than 20
indexTurnOn = df4.loc[abs(df4['TurnOn'] > 20),].index
df4.drop(indexTurnOn,inplace=True)
df4.info()

In [ ]:
df4['Mileage'].describe()

In [ ]:
plt.hist(df4['Mileage'], density=True, bins=20)
plt.show()

In [ ]:
df4.loc[df4['Mileage']>500,]['Mileage'].count()

In [ ]:
plt.hist(df4.loc[df4['Mileage']<=500]['Mileage'], density=True, bins=20)
# plt.savefig('figures/histogram-Mileage.png')
plt.show()

In [ ]:
### Delete entries with Mileage higher than 500
indexMileage = df4.loc[abs(df4['Mileage'] > 500),].index
df4.drop(indexMileage,inplace=True)
df4.reset_index(inplace=True)
df4.info()

In [ ]:
df4 = df4.drop('index',axis=1)
df4['liters_per_hour'].count()/df_backup.shape[0]

In [ ]:
# df4.to_excel("dataset/data_v1.xlsx", index=False)